In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Documents/mestrado/powerModelsExemplos`


# Leitura de modelo
O **PowerModelsDistribution** ou PMD não possui suporte nativo para adicionar um custo na bateria, portanto o repositório foi modificado localmente para inclusão dessa variável no modelo a partir de pequenas modificações.


Ao adicionar o custo do fonercimento de energia da bateria para o DSO é necessário também incluir a variável no problema de otimização.


O problema de otimização nativo do PMD se da por:
$$
\min \sum_{t \in \mathcal{T}} \left(\sum_{g \in \mathcal{G}} c_g \cdot P_{g, t} \right)
$$

sendo $\mathcal{T}$ o conjunto de cada instante de tempo, $t$ o indice do instante de tempo, $\mathcal{G}$ o conjunto de geradores, considerando PV e fonte de tensão, $c_{g}$ o custo de geração de cada um dos geradores e $P_{g, t}$ a potência injetada por cada gerador em determinado instante de tempo. Como o custo é constante para cada gerador neste exemplo, não é necessário indicar o indice do tempo.


A ideia do sistema em análise é criar um sistema que priorize o consumo local de energia. Para um teste inicial do favorecimento do consumo local de energia, pode-se considerar, que existe custo para injetar energia na rede, ou seja, um preço de venda negativo.

$$
\min \sum_{t \in \mathcal{T}} \left( \sum_{g \in \mathcal{G}} max({0, c_{s, g} \cdot P_{g, t})} + \sum_{b \in \mathcal{B}} max({0, c_{b} \cdot P_{b, t})} \right)
$$

onde $\mathcal{B}$ representa o conjunto dos sistemas de baterias, $c_{g}$ representa o custo de injeção de cada uma dos sistemas de baterias e $P_{b, t}$ a potencia injetada por cada sistema de baterias em determinado instante de tempo. A depender do custo de cada uma das fontes, a otimização será realizada dando prefência para as fontes mais baratas.

In [2]:
using PowerModelsDistribution
using Ipopt
using JuMP
using PowerPlots
using Plots

include("utils/load_data.jl")
results_path = "results/2025-04-21_solar_carga_armazenamento_opf_cost/"


########################
# construção de modelo #
########################

data_path = "1-MVLV-urban-5.303-1-no_sw"
load_data = get_load_data(data_path, 1, 1)

# coletando os dados de geração e mutiplicando para ser um valor próximo da carga
gen_data = get_gen_data(data_path, 1, 1) .* 20

eng_model = PowerModelsDistribution.parse_file("4Bus-DY-Bal/4Bus-DY-Bal.DSS")

solver = optimizer_with_attributes(
    Ipopt.Optimizer,
    "max_iter" => 20000,
    "tol" => 1e-10,)


[ PowerModelsDistribution | Info ] : Command 'calcvoltagebases' on line 36 in '4Bus-DY-Bal.DSS' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'solve' on line 37 in '4Bus-DY-Bal.DSS' is not supported, skipping.
[ PowerModelsDistribution | Info ] : basemva=100 is the default value, you may want to adjust sbase_default for better convergence


MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("max_iter") => 20000, MathOptInterface.RawOptimizerAttribute("tol") => 1.0e-10])

## Experimento com geração nominal de 750W para fonte fotovoltaica

In [3]:
#######################################
# adicionando série temporal de carga #
#######################################
# o passo da análise será de 15 minutos
# coletando 24horas de dados
time_indexes = Float64.(collect(1:96))


############################
# adicionando um novo bus #
############################
add_bus!(eng_model,
         "n5",
         rg=[0.0],
         grounded=[4],
         status=ENABLED,
         terminals=[1, 2, 3, 4],
         xg=[0.0])


##########################
# adicionando nova linha #
##########################
line3 = copy(eng_model["line"]["line2"])
line3["f_bus"] = "n3"
line3["t_bus"] = "n5"
eng_model["line"]["line3"] = line3


##########################
# adicionando nova carga #
##########################
add_load!(eng_model,
          "load2",
          "n5",
          [1, 2, 3, 4],
          pd_nom=[1800.0, 1800.0, 1800.0],
          configuration=WYE,
          status=ENABLED,
          vm_nom=2.40178,
          dispatchable=NO,
          qd_nom=[871.78, 871.78, 871.78])

# definindo a série temporal de carga ativa
# o parametro replace indica se a serie irá mutiplicar a potencia nominal ou substituir
# caso seja false, ela irá mutiplicar (caso a serie temporal esteja em PU)
pd_ts_l1 = Dict("time" => time_indexes,
                "values" => load_data.pload,
                "offset" => 0,
                "replace" => false)

# definindo a série temporal de carga reativa
qd_ts_l1 = Dict("time" => time_indexes,
                "values" => load_data.qload,
                "offset" => 0,
                "replace" => false)

# indicando a série temporal designada para carga load1
eng_model["time_series"] = Dict("pd_ts_l1" => pd_ts_l1, "qd_ts_l1" => qd_ts_l1)
eng_model["load"]["load1"]["time_series"] = Dict("pd_nom" => "pd_ts_l1",
                                                 "qd_nom" => "qd_ts_l1")

eng_model["time_series"] = Dict("pd_ts_l1" => pd_ts_l1, "qd_ts_l1" => qd_ts_l1)
eng_model["load"]["load2"]["time_series"] = Dict("pd_nom" => "pd_ts_l1",
                                                 "qd_nom" => "qd_ts_l1")


#########################################
# adicionando série temporal de geracao #
#########################################
# definindo que a série temporal é a mesma para cada uma das fases
gen_data_1 = []
for i in gen_data.pgen
    push!(gen_data_1, [i, i, i, 0])
end

# definindo a série temporal de geração
pd_ts_g1 = Dict("time" => time_indexes,
                "values" => gen_data_1,
                "offset" => 0,
                "replace" => false)

# adicionando um sistema fotovoltaico
add_solar!(eng_model,
           "pv1",
           "n4",
           configuration=WYE,
           [1, 2, 3, 4],
           pg=[250, 250, 250, 0],
           qg=[0, 0, 0, 0],
           pg_ub=[250, 250, 250, 0], # potencia ativa nominal
           pg_lb=[0, 0, 0, 0], # limite inferior de potencia ativa
           qg_ub=[0, 0, 0, 0], # potencia reativa nominal
           qg_lb=[0, 0, 0, 0]) # limite inferior de potencia reativa
eng_model["time_series"]["pd_ts_g1"] = pd_ts_g1
eng_model["solar"]["pv1"]["time_series"] = Dict("pg_ub" => "pd_ts_g1",
                                                "pg_lb" => "pd_ts_g1")

add_solar!(eng_model,
           "pv2",
           "n5",
           configuration=WYE,
           [1, 2, 3, 4],
           pg=[250, 250, 250, 0],
           qg=[0, 0, 0, 0],
           pg_ub=[250, 250, 250, 0], # potencia ativa nominal
           pg_lb=[0, 0, 0, 0], # limite inferior de potencia ativa
           qg_ub=[0, 0, 0, 0], # potencia reativa nominal
           qg_lb=[0, 0, 0, 0]) # limite inferior de potencia reativa
eng_model["time_series"]["pd_ts_g1"] = pd_ts_g1
eng_model["solar"]["pv2"]["time_series"] = Dict("pg_ub" => "pd_ts_g1",
                                                "pg_lb" => "pd_ts_g1")


#############################
# adicionando armazenamento #
#############################
add_storage!(eng_model,
             "bess_1",
             "n4",
             configuration=WYE,
             [1, 2, 3, 4],
             energy=40000,
             energy_ub=80000,
             charge_ub=7000,
             discharge_ub=7000,
             sm_ub=150000,
             cm_ub=1e6,
             qex=0,
             pex=0,
             charge_efficiency=100,
             discharge_efficiency=100,
             qs_ub=0,
             qs_lb=0,
             rs=0,
             xs=0)

add_storage!(eng_model,
             "bess_2",
             "n5",
             configuration=WYE,
             [1, 2, 3, 4],
             energy=40000,
             energy_ub=80000,
             charge_ub=7000,
             discharge_ub=7000,
             sm_ub=150000,
             cm_ub=1e6,
             qex=0,
             pex=0,
             charge_efficiency=100,
             discharge_efficiency=100,
             qs_ub=0,
             qs_lb=0,
             rs=0,
             xs=0)

eng_model["settings"]["sbase_default"] = 1000

# venda / compra
eng_model["voltage_source"]["source"]["cost_pg_parameters"] = [-100, 100]

eng_model["solar"]["pv1"]["cost_pg_parameters"] = [50, 50]
eng_model["solar"]["pv2"]["cost_pg_parameters"] = [50, 50]

eng_model["storage"]["bess_1"]["cost"] = [30, 50]
eng_model["storage"]["bess_2"]["cost"] = [30, 50]



2-element Vector{Int64}:
 30
 50

In [4]:
eng_model = make_multinetwork(eng_model) # indicando que se trata de uma série temporal
set_time_elapsed!(eng_model, 0.25) # indicando que o passo é de 15 minutos 
result = solve_mc_model(eng_model, ACPUPowerModel, solver, build_mc_mn_opf_cost; multinetwork=true)

[ PowerModelsDistribution | Info ] : assuming time is in hours for time_elapsed inference. if this is incorrect, manually adjust with set_time_elapsed!



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:   117184
Number of nonzeros in inequality constraint Jacobian.:    14400
Number of nonzeros in Lagrangian Hessian.............:   272388

Total number of variables............................:    18306
                     variables with only lower bounds:     2592
                variables with lower and upper bounds:     4194
                     variables with only upper bounds:        0
Total number of equality constraints.................:    17184
Total number of inequality c

Dict{String, Any} with 8 entries:
  "solve_time"         => 33.8263
  "optimizer"          => "Ipopt"
  "termination_status" => ALMOST_LOCALLY_SOLVED
  "dual_status"        => NEARLY_FEASIBLE_POINT
  "primal_status"      => NEARLY_FEASIBLE_POINT
  "objective"          => -5107.79
  "solution"           => Dict{String, Any}("nw"=>Dict{String, Dict{String, Any…
  "objective_lb"       => -Inf

In [5]:
########################
# avaliando resultados #
########################
line2_active_power = []
line3_active_power = []

load1_active_power = []
load2_active_power = []

pv1_active_power = []
pv2_active_power = []

bess_1_active_power = []
bess_2_active_power = []
bess_1_state = []
bess_2_state = []

bus_n3_1_voltage = []
bus_n3_2_voltage = []
bus_n3_3_voltage = []

for i in 1:96
    # line
    push!(line2_active_power, sum(result["solution"]["nw"]["$i"]["line"]["line2"]["pt"]))
    push!(line3_active_power, sum(result["solution"]["nw"]["$i"]["line"]["line3"]["pt"]))

    # load
    push!(load1_active_power, sum(result["solution"]["nw"]["$i"]["load"]["load1"]["pd"]))
    push!(load2_active_power, sum(result["solution"]["nw"]["$i"]["load"]["load2"]["pd"]))

    # solar
    push!(pv1_active_power, .- sum(result["solution"]["nw"]["$i"]["solar"]["pv1"]["pg"]))
    push!(pv2_active_power, .- sum(result["solution"]["nw"]["$i"]["solar"]["pv2"]["pg"]))

    # bess
    push!(bess_1_active_power, sum(result["solution"]["nw"]["$i"]["storage"]["bess_1"]["ps"]))
    push!(bess_1_state, result["solution"]["nw"]["$i"]["storage"]["bess_1"]["se"])
    push!(bess_2_active_power, sum(result["solution"]["nw"]["$i"]["storage"]["bess_2"]["ps"]))
    push!(bess_2_state, result["solution"]["nw"]["$i"]["storage"]["bess_2"]["se"])

    # bus
    push!(bus_n3_1_voltage, result["solution"]["nw"]["$i"]["bus"]["n4"]["vm"][1])
    push!(bus_n3_2_voltage, result["solution"]["nw"]["$i"]["bus"]["n4"]["vm"][2])
    push!(bus_n3_3_voltage, result["solution"]["nw"]["$i"]["bus"]["n4"]["vm"][3])

end

#######################################
# plotagem de graficos para validacao #
#######################################
time = time_indexes ./ 4
bar(time, load1_active_power + load2_active_power, label="Carga 1", linewidth=0.5)
bar!(time, pv1_active_power + pv2_active_power, label="Sistema FV", linewidth=0.5)
bar!(time, bess_1_active_power + bess_2_active_power, label="Armazenamento", linewidth=0.5)
bar!(time, line2_active_power + line3_active_power, label="Fluxo na Linha", linewidth=0.5)
title!("Potência Ativa dos Componentes")
xlabel!("Tempo (h)")
ylabel!("Potência (W)")
savefig(results_path * "potencia_ativa.png")

bar(time, pv1_active_power .+ pv2_active_power .+
          bess_1_active_power .+ bess_2_active_power .+
          load1_active_power .+ load2_active_power,
          label="Balanço no barramento", linewidth=0.5)
bar!(time, line2_active_power .+ line3_active_power, label="Fluxo na Linha", linewidth=0.5)
title!("Comparacao balanço e fluxo na linha")
xlabel!("Tempo (h)")
ylabel!("Potência (W)")
savefig(results_path * "validacao_linha.png")


plot(time, bess_1_active_power, 
    label="Potência Ativa (W)", 
    ylabel="Potência Ativa (W)",
    legend=:topright)
plot!(twinx(), time, bess_1_state, 
    label="Estado do Armazenamento (Wh)", 
    color=:red,
    ylabel="Estado de Carga (Wh)",
    legend=:topleft)
title!("Potência Ativa e Estado do Armazenamento")
savefig(results_path * "estado_armazenamento_ativo_1.png")
    

plot(time, bess_2_active_power, 
    label="Potência Ativa (W)", 
    ylabel="Potência Ativa (W)",
    legend=:topright)
plot!(twinx(), time, bess_2_state, 
    label="Estado do Armazenamento (Wh)", 
    color=:red,
    ylabel="Estado de Carga (Wh)",
    legend=:topleft)
title!("Potência Ativa e Estado do Armazenamento")
savefig(results_path * "estado_armazenamento_ativo_2.png")


plot(time, bus_n3_1_voltage, label="Tensão na Fase 1", linewidth=2)
plot!(time, bus_n3_2_voltage, label="Tensão na Fase 2", linewidth=2)
plot!(time, bus_n3_3_voltage, label="Tensão na Fase 3", linewidth=2)
title!("Tensão nas fases (barramento n3)")
xlabel!("Tempo (h)")
ylabel!("Tensão em Kv")
savefig(results_path * "tensao_barramento.png")
    
plot(time, gen_data.pgen .* (750), linewidth=2)
bar!(time, .- pv1_active_power, linewidth=2)
bar!(time, .- pv2_active_power, linewidth=2)
title!("Geração nominal solar")
xlabel!("Tempo (h)")
ylabel!("Tensão em Kv")
savefig(results_path * "geracao em PU.png")

"/home/luizfreire/Documents/mestrado/powerModelsExemplos/results/2025-04-21_solar_carga_armazenamento_opf_cost/geracao em PU.png"

## Experimento com geração nominal de 1500W para fonte fotovoltaica

In [6]:
eng_model = PowerModelsDistribution.parse_file("4Bus-DY-Bal/4Bus-DY-Bal.DSS")

#######################################
# adicionando série temporal de carga #
#######################################
# o passo da análise será de 15 minutos
# coletando 24horas de dados
time_indexes = Float64.(collect(1:96))


############################
# adicionando um novo bus #
############################
add_bus!(eng_model,
         "n5",
         rg=[0.0],
         grounded=[4],
         status=ENABLED,
         terminals=[1, 2, 3, 4],
         xg=[0.0])


##########################
# adicionando nova linha #
##########################
line3 = copy(eng_model["line"]["line2"])
line3["f_bus"] = "n3"
line3["t_bus"] = "n5"
eng_model["line"]["line3"] = line3


##########################
# adicionando nova carga #
##########################
add_load!(eng_model,
          "load2",
          "n5",
          [1, 2, 3, 4],
          pd_nom=[1800.0, 1800.0, 1800.0],
          configuration=WYE,
          status=ENABLED,
          vm_nom=2.40178,
          dispatchable=NO,
          qd_nom=[871.78, 871.78, 871.78])

# definindo a série temporal de carga ativa
# o parametro replace indica se a serie irá mutiplicar a potencia nominal ou substituir
# caso seja false, ela irá mutiplicar (caso a serie temporal esteja em PU)
pd_ts_l1 = Dict("time" => time_indexes,
                "values" => load_data.pload,
                "offset" => 0,
                "replace" => false)

# definindo a série temporal de carga reativa
qd_ts_l1 = Dict("time" => time_indexes,
                "values" => load_data.qload,
                "offset" => 0,
                "replace" => false)

# indicando a série temporal designada para carga load1
eng_model["time_series"] = Dict("pd_ts_l1" => pd_ts_l1, "qd_ts_l1" => qd_ts_l1)
eng_model["load"]["load1"]["time_series"] = Dict("pd_nom" => "pd_ts_l1",
                                                 "qd_nom" => "qd_ts_l1")

eng_model["time_series"] = Dict("pd_ts_l1" => pd_ts_l1, "qd_ts_l1" => qd_ts_l1)
eng_model["load"]["load2"]["time_series"] = Dict("pd_nom" => "pd_ts_l1",
                                                 "qd_nom" => "qd_ts_l1")


#########################################
# adicionando série temporal de geracao #
#########################################
# definindo que a série temporal é a mesma para cada uma das fases
gen_data_1 = []
for i in gen_data.pgen
    push!(gen_data_1, [i, i, i, 0])
end

# definindo a série temporal de geração
pd_ts_g1 = Dict("time" => time_indexes,
                "values" => gen_data_1,
                "offset" => 0,
                "replace" => false)

# adicionando um sistema fotovoltaico
add_solar!(eng_model,
           "pv1",
           "n4",
           configuration=WYE,
           [1, 2, 3, 4],
           pg=[500, 500, 500, 0],
           qg=[0, 0, 0, 0],
           pg_ub=[500, 500, 500, 0], # potencia ativa nominal
           pg_lb=[0, 0, 0, 0], # limite inferior de potencia ativa
           qg_ub=[0, 0, 0, 0], # potencia reativa nominal
           qg_lb=[0, 0, 0, 0]) # limite inferior de potencia reativa
eng_model["time_series"]["pd_ts_g1"] = pd_ts_g1
eng_model["solar"]["pv1"]["time_series"] = Dict("pg_ub" => "pd_ts_g1",
                                                "pg_lb" => "pd_ts_g1")

add_solar!(eng_model,
           "pv2",
           "n5",
           configuration=WYE,
           [1, 2, 3, 4],
           pg=[500, 500, 500, 0],
           qg=[0, 0, 0, 0],
           pg_ub=[500, 500, 500, 0], # potencia ativa nominal
           pg_lb=[0, 0, 0, 0], # limite inferior de potencia ativa
           qg_ub=[0, 0, 0, 0], # potencia reativa nominal
           qg_lb=[0, 0, 0, 0]) # limite inferior de potencia reativa
eng_model["time_series"]["pd_ts_g1"] = pd_ts_g1
eng_model["solar"]["pv2"]["time_series"] = Dict("pg_ub" => "pd_ts_g1",
                                                "pg_lb" => "pd_ts_g1")


#############################
# adicionando armazenamento #
#############################
add_storage!(eng_model,
             "bess_1",
             "n4",
             configuration=WYE,
             [1, 2, 3, 4],
             energy=40000,
             energy_ub=80000,
             charge_ub=7000,
             discharge_ub=7000,
             sm_ub=150000,
             cm_ub=1e6,
             qex=0,
             pex=0,
             charge_efficiency=100,
             discharge_efficiency=100,
             qs_ub=0,
             qs_lb=0,
             rs=0,
             xs=0)

add_storage!(eng_model,
             "bess_2",
             "n5",
             configuration=WYE,
             [1, 2, 3, 4],
             energy=40000,
             energy_ub=80000,
             charge_ub=7000,
             discharge_ub=7000,
             sm_ub=150000,
             cm_ub=1e6,
             qex=0,
             pex=0,
             charge_efficiency=100,
             discharge_efficiency=100,
             qs_ub=0,
             qs_lb=0,
             rs=0,
             xs=0)

eng_model["settings"]["sbase_default"] = 1000

# venda / compra
eng_model["voltage_source"]["source"]["cost_pg_parameters"] = [-100, 100]

eng_model["solar"]["pv1"]["cost_pg_parameters"] = [50, 50]
eng_model["solar"]["pv2"]["cost_pg_parameters"] = [50, 50]

eng_model["storage"]["bess_1"]["cost"] = [30, 50]
eng_model["storage"]["bess_2"]["cost"] = [30, 50]


[ PowerModelsDistribution | Info ] : Command 'calcvoltagebases' on line 36 in '4Bus-DY-Bal.DSS' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'solve' on line 37 in '4Bus-DY-Bal.DSS' is not supported, skipping.
[ PowerModelsDistribution | Info ] : basemva=100 is the default value, you may want to adjust sbase_default for better convergence


2-element Vector{Int64}:
 30
 50

In [7]:
eng_model = make_multinetwork(eng_model) # indicando que se trata de uma série temporal
set_time_elapsed!(eng_model, 0.25) # indicando que o passo é de 15 minutos 
result = solve_mc_model(eng_model, ACPUPowerModel, Ipopt.Optimizer, build_mc_mn_opf_cost; multinetwork=true)

[ PowerModelsDistribution | Info ] : assuming time is in hours for time_elapsed inference. if this is incorrect, manually adjust with set_time_elapsed!


This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:   117184
Number of nonzeros in inequality constraint Jacobian.:    14400
Number of nonzeros in Lagrangian Hessian.............:   272388

Total number of variables............................:    18306
                     variables with only lower bounds:     2592
                variables with lower and upper bounds:     4194
                     variables with only upper bounds:        0
Total number of equality constraints.................:    17184
Total number of inequality constraints...............:     5952
        inequality constraints with only lower bounds:     2016
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     3936

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -3.7943910e+03 5.24e+00 5.00e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

Dict{String, Any} with 8 entries:
  "solve_time"         => 120.437
  "optimizer"          => "Ipopt"
  "termination_status" => OTHER_ERROR
  "dual_status"        => UNKNOWN_RESULT_STATUS
  "primal_status"      => FEASIBLE_POINT
  "objective"          => -6276.95
  "solution"           => Dict{String, Any}("nw"=>Dict{String, Dict{String, Any…
  "objective_lb"       => -Inf

In [8]:
########################
# avaliando resultados #
########################
line2_active_power = []
line3_active_power = []

load1_active_power = []
load2_active_power = []

pv1_active_power = []
pv2_active_power = []

bess_1_active_power = []
bess_2_active_power = []
bess_1_state = []
bess_2_state = []

bus_n3_1_voltage = []
bus_n3_2_voltage = []
bus_n3_3_voltage = []

for i in 1:96
    # line
    push!(line2_active_power, sum(result["solution"]["nw"]["$i"]["line"]["line2"]["pt"]))
    push!(line3_active_power, sum(result["solution"]["nw"]["$i"]["line"]["line3"]["pt"]))

    # load
    push!(load1_active_power, sum(result["solution"]["nw"]["$i"]["load"]["load1"]["pd"]))
    push!(load2_active_power, sum(result["solution"]["nw"]["$i"]["load"]["load2"]["pd"]))

    # solar
    push!(pv1_active_power, .- sum(result["solution"]["nw"]["$i"]["solar"]["pv1"]["pg"]))
    push!(pv2_active_power, .- sum(result["solution"]["nw"]["$i"]["solar"]["pv2"]["pg"]))

    # bess
    push!(bess_1_active_power, sum(result["solution"]["nw"]["$i"]["storage"]["bess_1"]["ps"]))
    push!(bess_1_state, result["solution"]["nw"]["$i"]["storage"]["bess_1"]["se"])
    push!(bess_2_active_power, sum(result["solution"]["nw"]["$i"]["storage"]["bess_2"]["ps"]))
    push!(bess_2_state, result["solution"]["nw"]["$i"]["storage"]["bess_2"]["se"])

    # bus
    push!(bus_n3_1_voltage, result["solution"]["nw"]["$i"]["bus"]["n4"]["vm"][1])
    push!(bus_n3_2_voltage, result["solution"]["nw"]["$i"]["bus"]["n4"]["vm"][2])
    push!(bus_n3_3_voltage, result["solution"]["nw"]["$i"]["bus"]["n4"]["vm"][3])

end

#######################################
# plotagem de graficos para validacao #
#######################################
time = time_indexes ./ 4
bar(time, load1_active_power + load2_active_power, label="Carga 1", linewidth=0.5)
bar!(time, pv1_active_power + pv2_active_power, label="Sistema FV", linewidth=0.5)
bar!(time, bess_1_active_power + bess_2_active_power, label="Armazenamento", linewidth=0.5)
bar!(time, line2_active_power + line3_active_power, label="Fluxo na Linha", linewidth=0.5)
title!("Potência Ativa dos Componentes")
xlabel!("Tempo (h)")
ylabel!("Potência (W)")
savefig(results_path * "potencia_ativa high.png")

bar(time, pv1_active_power .+ pv2_active_power .+
          bess_1_active_power .+ bess_2_active_power .+
          load1_active_power .+ load2_active_power,
          label="Balanço no barramento", linewidth=0.5)
bar!(time, line2_active_power .+ line3_active_power, label="Fluxo na Linha", linewidth=0.5)
title!("Comparacao balanço e fluxo na linha")
xlabel!("Tempo (h)")
ylabel!("Potência (W)")
savefig(results_path * "validacao_linha high.png")


plot(time, bess_1_active_power, 
    label="Potência Ativa (W)", 
    ylabel="Potência Ativa (W)",
    legend=:topright)
plot!(twinx(), time, bess_1_state, 
    label="Estado do Armazenamento (Wh)", 
    color=:red,
    ylabel="Estado de Carga (Wh)",
    legend=:topleft)
title!("Potência Ativa e Estado do Armazenamento")
savefig(results_path * "estado_armazenamento_ativo_1 high.png")
    

plot(time, bess_2_active_power, 
    label="Potência Ativa (W)", 
    ylabel="Potência Ativa (W)",
    legend=:topright)
plot!(twinx(), time, bess_2_state, 
    label="Estado do Armazenamento (Wh)", 
    color=:red,
    ylabel="Estado de Carga (Wh)",
    legend=:topleft)
title!("Potência Ativa e Estado do Armazenamento")
savefig(results_path * "estado_armazenamento_ativo_2 high.png")


plot(time, bus_n3_1_voltage, label="Tensão na Fase 1", linewidth=2)
plot!(time, bus_n3_2_voltage, label="Tensão na Fase 2", linewidth=2)
plot!(time, bus_n3_3_voltage, label="Tensão na Fase 3", linewidth=2)
title!("Tensão nas fases (barramento n3)")
xlabel!("Tempo (h)")
ylabel!("Tensão em Kv")
savefig(results_path * "tensao_barramento high.png")
    
plot(time, gen_data.pgen .* (1500), linewidth=2)
bar!(time, .- pv1_active_power, linewidth=2)
bar!(time, .- pv2_active_power, linewidth=2)
title!("Geração nominal solar")
xlabel!("Tempo (h)")
ylabel!("Tensão em Kv")
savefig(results_path * "geracao em PU high.png")

"/home/luizfreire/Documents/mestrado/powerModelsExemplos/results/2025-04-21_solar_carga_armazenamento_opf_cost/geracao em PU high.png"

In [9]:
source = []
for i in 1:96
    push!(source, sum(result["solution"]["nw"]["$i"]["voltage_source"]["source"]["pg"]))
end

In [10]:
sum(+ 50 * (pv1_active_power + pv2_active_power)
    + 50 * max.(0, bess_1_active_power + bess_2_active_power)
    + 30 * min.(0, bess_1_active_power + bess_2_active_power)
    + 100 * max.(0, source)
    - 100 * min.(0, source))/ 1000

-6277.145472718572

In [11]:
sum(source)

1965.0404332516466

In [12]:
eng_model["nw"]["1"]["settings"]["power_scale_factor"]

1000.0

In [13]:
transform_data_model(eng_model)["nw"]["50"]["gen"]["3"]

Dict{String, Any} with 21 entries:
  "pg"            => [0.0, 0.0, 0.0]
  "model"         => 2
  "connections"   => [1, 2, 3]
  "shutdown"      => 0.0
  "startup"       => 0.0
  "configuration" => WYE
  "name"          => "_virtual_gen.voltage_source.source"
  "qg"            => [0.0, 0.0, 0.0]
  "gen_bus"       => 10
  "pmax"          => [Inf, Inf, Inf]
  "vbase"         => 7.19956
  "source_id"     => "voltage_source.source"
  "vg"            => [0.001, 0.001, 0.001]
  "index"         => 3
  "cost"          => [-100, 100]
  "gen_status"    => 1
  "qmax"          => [Inf, Inf, Inf]
  "qmin"          => [-Inf, -Inf, -Inf]
  "control_mode"  => 1
  "pmin"          => [-Inf, -Inf, -Inf]
  "ncost"         => 2